## Set-up environment

In [1]:
!git clone -b add_mixin https://github.com/nielsrogge/blt
%cd blt
!pip install -r requirements.txt

Cloning into 'blt'...
remote: Enumerating objects: 538, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 538 (delta 148), reused 102 (delta 87), pack-reused 334 (from 2)
Receiving objects: 100% (538/538), 476.65 KiB | 13.24 MiB/s, done.
Resolving deltas: 100% (305/305), done.
/content/blt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypro

In [1]:
!pip install --upgrade huggingface_hub

## Load model

In [3]:
%cd /content/blt

/content/blt


In [5]:
!pip install .

Processing /content/blt
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.1 MB/s eta 0:00:00
  Created wheel for bytelatent: filename=bytelatent-0.1.0-py3-none-any.whl size=147313 sha256=91694552c716b94b6dffefe2c3eb920dd4b9d89f73aaa54f7048e1498c0655a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-9gg36u5t/wheels/10/13/27/f0c4bd462820770c83979f05849abafc28c4fe62dcba1c6585
Successfully built bytelatent


In [4]:
import xformers
from bytelatent.generate import load_consolidated_model_and_tokenizer
from huggingface_hub import hf_hub_download
import torch


checkpoint_path = hf_hub_download(repo_id="facebook/blt-1b", filename="consolidated.pth")

model, tokenizer, train_cfg = load_consolidated_model_and_tokenizer(checkpoint_path)

AttributeError: '_OpNamespace' 'xformers' object has no attribute 'efficient_attention_forward_cutlass'

## Push to the hub

In [ ]:
model.push_to_hub("facebook/blt-1b")

## Reload

Now anyone can use the model like so:

In [5]:
from bytelatent.model.blt import ByteLatentTransformer

model = ByteLatentTransformer.from_pretrained("facebook/blt-1b")

AttributeError: '_OpNamespace' 'xformers' object has no attribute 'efficient_attention_forward_cutlass'

In [6]:
import os

import torch
import typer

from bytelatent.distributed import DistributedArgs, setup_torch_distributed
from bytelatent.generate import load_consolidated_model_and_tokenizer
from bytelatent.generate_blt import generate_nocache
from bytelatent.model.blt import ByteLatentTransformer
from bytelatent.tokenizers.blt_tokenizer import BltTokenizer


def main(prompt: str, model_name: str = "blt-1b"):
    distributed_args = DistributedArgs()
    distributed_args.configure_world()
    if not torch.distributed.is_initialized():
        setup_torch_distributed(distributed_args)
    checkpoint_path = os.path.join("hf-weights", model_name)
    print(f"Loading BLT model: {model_name}")
    model, tokenizer, train_cfg = load_consolidated_model_and_tokenizer(
        checkpoint_path,
    )
    assert isinstance(model, ByteLatentTransformer)
    assert isinstance(tokenizer, BltTokenizer)
    patcher_args = train_cfg.data.patcher_args.model_copy(deep=True)
    patcher_args.realtime_patching = True
    print("Loading entropy model and patcher")
    patcher_args.entropy_model_checkpoint_dir = os.path.join(
        checkpoint_path, "entropy_model"
    )
    patcher = patcher_args.build()
    prompts = [prompt]
    outputs = generate_nocache(
        prompts, model=model, tokenizer=tokenizer, patcher=patcher
    )
    text_outputs = [tokenizer.decode(t) for t in outputs]
    for p, t in zip(prompts, text_outputs):
        print(f'Prompt: "{p}" Completion: "{t}"')
        print()


if __name__ == "__main__":
    typer.run(main)


AttributeError: '_OpNamespace' 'xformers' object has no attribute 'efficient_attention_forward_cutlass'